In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Concatenate, GlobalAveragePooling2D, Dense, Dropout, LSTM, Reshape

# Define the input shape
input_shape = (None, None, 3) # Change this according to your data

# Define the input layer
input_layer = Input(shape=input_shape)

# Apply 1x16 convolutional layer stride 1 with depth of 64
conv1 = Conv2D(filters=64, kernel_size=(1, 16), strides=1, padding='same')(input_layer)

# Apply 3x3 convolutional layer stride 1
conv2 = Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same')(conv1)

# Apply this output to 4 different layers parallely of size 1x3, 3x1, 1x5, 5x1
conv3_1 = Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same')(conv2)
conv3_2 = Conv2D(filters=64, kernel_size=(3, 1), strides=1, padding='same')(conv2)
conv3_3 = Conv2D(filters=64, kernel_size=(1, 5), strides=1, padding='same')(conv2)
conv3_4 = Conv2D(filters=64, kernel_size=(5, 1), strides=1, padding='same')(conv2)

# Add all four output from this
conv4 = Concatenate()([conv3_1, conv3_2, conv3_3, conv3_4])

# Apply maxpool then batchnormalisation then relu
conv5 = MaxPooling2D(pool_size=(2, 2))(conv4)
conv6 = BatchNormalization()(conv5)
conv7 = Activation('relu')(conv6)

# Apply 1x7 convolution depth 64 and 7x1 convolutional layer stride1 depth 64 then batch normalization then softmax
conv8 = Conv2D(filters=64, kernel_size=(1, 7), strides=1, padding='same')(conv7)
conv9 = Conv2D(filters=64, kernel_size=(7, 1), strides=1, padding='same')(conv8)
conv10 = BatchNormalization()(conv9)
conv11 = Activation('softmax')(conv10)

# Apply 5x1 convolutional layer stride1 then 3x1 convolutional layer stride1 then 5x1 convolutional layer stride1 then 3x1 convolutional layer stride
conv12 = Conv2D(filters=64, kernel_size=(5, 1), strides=1)(conv11)
conv13 = Conv2D(filters=64, kernel_size=(3, 1), strides=1)(conv12)
conv14 = Conv2D(filters=64, kernel_size=(5, 1), strides=1)(conv13)
conv15 = Conv2D(filters=64, kernel_size=(3, 1), strides=1)(conv14)

# Flatten it apply global average pool then do depth wise convolution flatten in feed to fc1 and then put dropout 0.2 then apply fc2 then again dropout 0.2 then again fc3

# Remove the Reshape layer before the conv2d_49 layer
# reshape = Reshape((-1))(conv15)

dw_conv = Conv2D(filters=64, kernel_size=(3, 3), strides=1)(conv15)
dw_flat = GlobalAveragePooling2D()(dw_conv)

# Add LSTM layer
dw_flat = GlobalAveragePooling2D()(dw_conv)
dw_reshaped = Reshape((1, 64))(dw_flat)  
lstm_layer = LSTM(units=64)(dw_reshaped)

fc1 = Dense(units=128)(lstm_layer)
drop1 = Dropout(rate=0.2)(fc1)
fc2 = Dense(units=64)(drop1)
drop2 = Dropout(rate=0.2)(fc2)
fc3 = Dense(units=10)(drop2) # Change this according to your number of classes

# Define the output layer
output_layer = Activation('softmax')(fc3)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Print the model summary
model.summary()
model.save("Sept2.h5")